In [69]:
import camelot
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter    
from openpyxl import Workbook

In [70]:
pdf_file_name = 'test2.pdf'
excel_file_name = 'extracted_tables.xlsx'
bank_name = 'Bank1'
table_columns = ['DATE', 'NARRATION', 'CHQ.NO.', 'WITHDRAWAL (DR)', 'DEPOSIT (CR)', 'BALANCE']

In [33]:
# Bank 0
tables = camelot.read_pdf(pdf_file_name, pages='all', flavor='stream')

Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)


In [71]:
# Bank 1
tables = camelot.read_pdf(pdf_file_name, pages='all', flavor='lattice', strip_text='\n')

Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)
Ignoring wrong pointing object 976 0 (offset 0)


In [72]:
print(tables)

<TableList n=4>


In [28]:
# Create a new csv file and write tables to separate csv files
for index, table in enumerate(tables):
    table.df.to_csv('table_' + str(index) + '.csv', index=False)

In [74]:
# Create a new Excel workbook and write tables to separate sheets
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
    for index, table in enumerate(tables):
        # Convert table to DataFrame
        df = table.df
        
        # Write each DataFrame to a different sheet
        sheet_name = f'Table_{index + 1}'
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Access the worksheet through writer.sheets
        worksheet = writer.sheets[sheet_name]
        
        # Adjust column widths
        for col_idx, col_cells in enumerate(df.columns, start=1):
            max_length = max(df[col_cells].astype(str).apply(len).max(), col_cells)
            adjusted_width = max_length + 2  # Adding some padding
            worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width

print(f"All tables have been extracted and saved to {excel_file_name}.")

All tables have been extracted and saved to extracted_tables.xlsx.


## Bank 0

In [5]:
user_name = 'SAI DHEERAJ PEKETI'

In [36]:
import re

def is_valid_date(text):
    date_pattern1 = r'^\d{2}-\d{2}-\d{4}$'
    date_pattern2 = r'^\d{2}-\d{2}-\d{2}$'
    return bool(re.match(date_pattern1, text)) or bool(re.match(date_pattern2, text))

def is_valid_date_or_empty(text):
    date_pattern = r'^\d{2}-\d{2}-\d{4}$'
    return bool(re.match(date_pattern, text)) or text == ''

In [37]:
new_rows = []
for index, table in enumerate(tables):
    if table.df.iloc[0, 0] == user_name:
        print(f'Table_{index + 1}: {table.df.shape[1]}')
        current_table_df = table.df.iloc[2:].reset_index(drop=True)
        print(current_table_df.columns)
        if table.df.shape[1] == 7:
            current_table_df.loc[current_table_df[1] != '', 2] = current_table_df[1]
            current_table_df.drop([1], axis=1, inplace=True)
        elif table.df.shape[1] == 6:
            invalid_dates = ~current_table_df[0].apply(is_valid_date_or_empty)
            current_table_df.loc[invalid_dates, 1] = current_table_df.loc[invalid_dates, 0]
            current_table_df.loc[invalid_dates, 0] = ''
        current_table_df.reset_index(drop=True, inplace=True)
        current_table_df.columns = table_columns
        isFirst = False
        i = 0
        
        while i < len(current_table_df):
            if is_valid_date(current_table_df.iloc[i, 0]):
                isFirst = True
                new_rows.append({
                    'DATE': current_table_df.iloc[i, 0],
                    'NARRATION': current_table_df.iloc[i, 1],
                    'CHQ.NO.': current_table_df.iloc[i, 2],
                    'WITHDRAWAL (DR)': current_table_df.iloc[i, 3],
                    'DEPOSIT (CR)': current_table_df.iloc[i, 4],
                    'BALANCE': current_table_df.iloc[i, 5]
                })
                i += 1
            else:
                if i + 1 < len(current_table_df) and i + 2 < len(current_table_df):
                    new_rows.append({
                        'DATE': current_table_df.iloc[i + 1, 0],
                        'NARRATION': current_table_df.iloc[i, 1]  + current_table_df.iloc[i + 2, 1],
                        'CHQ.NO.': current_table_df.iloc[i + 1, 2],
                        'WITHDRAWAL (DR)': current_table_df.iloc[i + 1, 3],
                        'DEPOSIT (CR)': current_table_df.iloc[i + 1, 4],
                        'BALANCE': current_table_df.iloc[i + 1, 5]
                    })
                i += 3
if len(new_rows) != 0:
    table_df = pd.DataFrame(new_rows)
    table_df['WITHDRAWAL (DR)'] = pd.to_numeric(table_df['WITHDRAWAL (DR)'], errors='coerce')
    table_df['DEPOSIT (CR)'] = pd.to_numeric(table_df['DEPOSIT (CR)'], errors='coerce')
    
    # Write to Excel file
    file_name = 'table.xlsx'
    table_df.to_excel(file_name, index=False, engine='openpyxl')
    workbook = load_workbook(file_name)
    worksheet = workbook.active
    
    for col_idx, col_name in enumerate(table_df.columns, start=1):
        max_length = max(
            table_df[col_name].astype(str).apply(len).max(),
            len(col_name)
        )
        adjusted_width = max_length + 2  # Add padding
        worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width
    
    workbook.save(file_name)

## Bank 1

In [93]:
def extract_opening_balance_info(text):
    pattern = r"Your Opening Balance on (\d{2}-\d{2}-\d{2}):\s*([\d,.]+)"
    match = re.search(pattern, text)
    if match:
        description = f"Your Opening Balance"
        date = match.group(1)
        amount = match.group(2)
        return True, date, description, amount
    else:
        return False, None, None, None
    
def extract_closing_balance_info(text):
    pattern = r"Your Closing Balance on (\d{2}-\d{2}-\d{2}):([\d,.]+)"
    match = re.search(pattern, text)
    if match:
        date = match.group(1) 
        amount = match.group(2) 
        converted_text = f"Your Closing Balance on {date}:      {amount}"
        return converted_text
    else:
        return None

In [96]:
new_rows = []
for index, table in enumerate(tables):
    if index >= 3:
        print(f'Table_{index + 1}: {table.df.shape[0]}')
        isOpening, date, description, amount = extract_opening_balance_info(table.df.iloc[0, 0])
        if isOpening:
            new_rows.append({
                'DATE': description + ' on ' + str(date) + ':      ' + str(amount),
                'NARRATION': '',
                'CHQ.NO.': '',
                'WITHDRAWAL (DR)': '',
                'DEPOSIT (CR)': '',
                'BALANCE': ''
            })
        current_table_df = table.df.iloc[1:].reset_index(drop=True)
        print(current_table_df.columns)
        table_columns = ['DATE', 'NARRATION', 'CHQ.NO.', 'WITHDRAWAL (DR)', 'DEPOSIT (CR)', 'BALANCE']
        current_table_df.reset_index(drop=True, inplace=True)
        current_table_df.columns = table_columns
        i = 0
        while i < len(current_table_df):
            if i == len(current_table_df) - 1:
                text = extract_closing_balance_info(table.df.iloc[i+1, 0])
                if text:
                    new_rows.append({
                        'DATE': text,
                        'NARRATION': '',
                        'CHQ.NO.': '',
                        'WITHDRAWAL (DR)': '',
                        'DEPOSIT (CR)': '',
                        'BALANCE': ''
                    })
                    break
            new_rows.append({
                'DATE': current_table_df.iloc[i, 0],
                'NARRATION': current_table_df.iloc[i, 1],
                'CHQ.NO.': current_table_df.iloc[i, 2],
                'WITHDRAWAL (DR)': current_table_df.iloc[i, 3],
                'DEPOSIT (CR)': current_table_df.iloc[i, 4],
                'BALANCE': current_table_df.iloc[i, 5]
            })
            i += 1
        

if len(new_rows) != 0:
    table_df = pd.DataFrame(new_rows)
    table_df.columns = table_columns
    table_df['WITHDRAWAL (DR)'] = pd.to_numeric(table_df['WITHDRAWAL (DR)'], errors='coerce')
    table_df['DEPOSIT (CR)'] = pd.to_numeric(table_df['DEPOSIT (CR)'], errors='coerce')
    
    # Write to Excel file
    file_name = 'table.xlsx'
    table_df.to_excel(file_name, index=False, engine='openpyxl')
    workbook = load_workbook(file_name)
    worksheet = workbook.active
    
    for col_idx, col_name in enumerate(table_df.columns, start=1):
        max_length = max(
            table_df[col_name].astype(str).apply(len).max(),
            len(col_name)
        )
        adjusted_width = max_length + 2  # Add padding
        worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width
    
    workbook.save(file_name)

Table_4: 6
RangeIndex(start=0, stop=6, step=1)
